In [ ]:
# default_exp train

In [ ]:
#hide
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#hide 
# %load_ext autoreload 
# %autoreload 2
%cd "/content/drive/MyDrive/Coding/ModelAssistedLabel"

/content/drive/MyDrive/Coding/ModelAssistedLabel


# Training Models
> wrapping `yolov5/train.py`

We're building towards generating a model with a single call.

`Trainer` is a fairly bare-bones wrapper built around `train.py`.

`AutoWeights` is a more robust wrapper and allows for custom naming/placement of the results folder.

In [ ]:
# export
from ModelAssistedLabel.config import Defaults
import os

class Trainer():
  """A wrapper for Ultralytic's `test.py`
  
  Write the backbone of the model to file and then run YOLOv5's train file."""

  def __init__(self, name, yaml_file = "models/custom_yolov5s.yaml"):
    """
    sets the current directory to the project's root as defined in Defaults.

    Args:
      name: identifier for results
      yaml_file: path to write the file 
    """
    os.chdir(Defaults().root)
    self.yaml_file = yaml_file
    self.name = name
    self.template = Defaults().trainer_template

  def write_yaml(self):
    """
    Records YOLOv5 architecture
    """
    f = open(f"yolov5/{self.yaml_file}","w+")
    f.writelines(self.template)
    f.close()

  def train(self, epochs):
    """
    wrapper for train.py.

    Args:
      epochs: number of iterations
    """
    self.write_yaml()
    os.chdir("yolov5")
    os.system("pip install -r requirements.txt")
    os.system(f"python train.py --img 416 --batch 16 --epochs {epochs} --data '../data.yaml' --cfg {self.yaml_file} --weights '' --name {self.name}  --cache")
    os.chdir("..")

In [ ]:
!nbdev_build_lib

Converted 00_config.ipynb.
Converted 01_split.ipynb.
Converted 02_train.ipynb.
Converted 03_detect.ipynb.
Converted index.ipynb.


In [ ]:
# export
from ModelAssistedLabel.train import Trainer
from ModelAssistedLabel.fileManagement import Generation
from datetime import datetime

class AutoWeights():
  """Given a bag of images (.jpg) and labels (.txt) in YOLOv5 format in a repository,
  initialize the ROOT directory with a train-valid-test split and a file needed 
  by the Ultralytics repository. Pairs are identified if `image[:-4] == label[:-4]`

  Then call `generate_weights` to run `train.py`. The resultant file will try to 
  be moved to the `out_dir` and if a conflict exists, a new name will be made.
  """
  def __init__(self, origin_images_dir, name="AutoWeight <name>", out_dir=".", MAX_SIZE=5, custom_data_yaml=None, verbose=True, train_path = "yolov5/runs/train"):
    """
    Args:
      origin_images_dir: location of the bag of images/labels
      out_dir: where the results of train.py are moved
      MAX_SIZE: parameter for `Generation`
      custom_data_yaml: see `Defaults`'s `data_yaml` for the default value
      verbose: Print summary information
      train_path: path to Ultralytic's default output folder
    """
    self.resource_paths = ["test/", "train/", "valid/"]
    self.origin_images_dir = origin_images_dir
    self.name = name
    self.out_dir = out_dir
    self.train_path = train_path
    self.custom_data_yaml = custom_data_yaml
    self.g = Generation(repo=self.origin_images_dir, out_dir=self.out_dir, data_yaml=self.custom_data_yaml)
    
    for r in self.resource_paths:
      if os.path.exists(r):
        listdir = len(os.listdir(r))
      else:
        listdir = "n/a"

      print('Directory:', r, "|" , str(listdir),"files")
      found = []

    for r in self.resource_paths: #make sure none of these paths already exist
      if os.path.exists(r):
        found.append(r)
    

  def generate_weights(self, epochs, tidy_weights=True, MAX_SIZE=None):
    """
    Creates a `Trainer` object and trains for a given amount of time.

    Args:
      epochs: number of iterations (according to docs, over 3000 is not uncommon)
      tidy_weights: if True, remove all of the resources in `self.resources`
    
    Returns:
      path to the output folder of train.py
    """
    override = True
    if len(found) > 0:
      override = len(input(f"Folders '{found}' exists. Press 'Enter' to leave the files alone, enter anything at all to potentially overwrite")) > 0
      
    #automatically build the resource paths and prepare for traniing
    self.__prepare_split__(MAX_SIZE=MAX_SIZE, data_yaml=self.custom_data_yaml, verbose=self.verbose, override=override)

    t = Trainer(self.name)
    ldir = lambda path: set(os.listdir(path))

    before = ldir(self.train_path)
    t.train(epochs)
    after = ldir(self.train_path)
    
    assert len(after) == len(before)+1 #only should have made one new file
    diff = list(after - before)[0]

    results_path = os.path.join(self.train_path, diff)

    if tidy_weights:
      results_path = self.__tidy_weights__(results_path = results_path)

    self.__cleanup__()
    self.last_results_path = results_path
    return results_path

  def __prepare_split__(self, MAX_SIZE, data_yaml, verbose, override):
    """
    Gets the local filesystem ready to run the wrapper for "train.py".

    Args:
      MAX_SIZE:
      data_yaml:
      verbose: print summary information for the split
    """  
    if data_yaml is None:
      data_yaml = Defaults().data_yaml
    
    if override:
      self.g.set_split(MAX_SIZE=MAX_SIZE)
      self.__split_and_organize_folders__(verbose=verbose)
    else:
      self.g.set_split_from_disk()
  
  def __split_and_organize_folders__(self, verbose):
    """
    Assume zip file contains 4 items:
      * data.yaml
      * train/
      * valid/
      * test/

    Extract these 4 resources to the ROOT directory and remove the original 
    part of the file.

    Args:
      verbose: print summary information about the split
    """
    assert self.g is not None
    assert self.g.split is not None

    def tostr(split):
      return [{k: len(v)} for k,v in split.items()]
    def sumg(split):
      return sum([list(x.values())[0] for x in tostr(split)])
    if verbose:
      print("summary: ", tostr(self.g.split))
      print("checksum:", sumg(self.g.split))

    zipped = self.g.write_split_to_disk(self.name) #create a zip file in the ROOT directory
    local = os.path.basename(zipped)
    os.system(f'unzip "{local}"') #grab data
    #move the contents of the zip file into postion within the ROOT directory
    for content in self.resource_paths:
      os.system(f"mv '{local[:-4]}/{content}' .")
    #remove zip file
    os.system(f"rm -f -r '{local}'")
    #removed the folder that was taken out of the zip
    os.system(f"rm -f -r '{local[:-4]}'")

  def __cleanup__(self):
    """
    Removes all resources in `self.resource_paths` from the filesystem.
    """
    for r in self.resource_paths:
      os.system(f"rm -f -r {r}")

  def __tidy_weights__(self, results_path):
    """
    Moves the results to a desired directly while ensuring that no data is overwritten

    Args:
      results_path: path to the folder that has desired information
    
    Returns:
      Path to the newly-moved results
    """      
    while True:
      now = datetime.now()
      out = f"{os.path.basename(results_path)}-{now.strftime('%f')}" #basename + "random" string
      outfolder = os.path.join(self.out_dir, out)
      if not os.path.exists(outfolder): #only stops if a unique name has been found
        break
    os.system(f"mv '{results_path}' '{outfolder}'")
    return outfolder

In [ ]:
aw = AutoWeights("./Image Repo/labeled/Final Roboflow Export (841)")

Directory: test/ | n/a files
Directory: train/ | n/a files
Directory: valid/ | n/a files


In [ ]:
aw.g.split  